In [1]:
import pandas as pd
import os
from datetime import datetime
from datetime import date

from sqlalchemy import create_engine
from config import engine
from openpyxl import load_workbook

usage_file = 'TestData/Eq Usage Post - Test.xlsx'

df_read = pd.read_excel(usage_file)
df_extr = df_read[[ 'equipment_no', 'transaction_no','date_booked', 'job_no', 'equipment_units', 'cost_code_no']].copy()

today = date.today()
df_extr['date_modified'] = today
print(df_extr)


  equipment_no  transaction_no date_booked  job_no  equipment_units  \
0   E39                   1344  2024-11-15  231648             3.00   
1   P91                   1344  2024-11-15  231659             6.50   
2   P91                   1344  2024-11-15  231659             1.50   
3   P101                  1344  2024-11-15  231664             5.00   
4   P76                   1344  2024-11-17  231675             0.01   

   cost_code_no date_modified  
0       6521.03    2025-01-26  
1       1001.00    2025-01-26  
2       3030.05    2025-01-26  
3       1001.00    2025-01-26  
4       1001.00    2025-01-26  


In [2]:
df_extr['job_no'] = df_extr['job_no'].astype(str)

df_extr['cost_code_no'] = df_extr['cost_code_no'].astype(str)
df_extr['cost_code_no'] = df_extr['cost_code_no'].apply(lambda x: x.replace(' ', ''))

df_extr['equipment_no'] = df_extr['equipment_no'].apply(lambda x: x.replace(' ', ''))

df_extr['date_booked'] = df_extr['date_booked'].dt.date






#df_extr.to_sql('eq_usage', engine, if_exists='append', index=False)

In [3]:
df_extg = pd.read_sql_table('eq_usage', engine)
#df_extg['job_no'] = df_extg['job_no'].astype(str)
df_extg['cost_code_no'] = df_extg['cost_code_no'].astype(str)

df_extg['date_modified'] = df_extg['date_modified'].dt.date
df_extg['date_booked'] = df_extg['date_booked'].dt.date
df_extg.drop('id', axis=1, inplace=True)
df_extg.drop('date_booked', axis=1, inplace=True)
df_extr.drop('date_booked', axis=1, inplace=True)

# df_add = df_extr.drop_duplicates(subset=df_extg.columns.tolist(), keep=False)
#df_add = df_extr[~df_extr[['equipment_no']].isin(df_extg[['equipment_no']]).all(axis=1)]
# print(len(df_add))
df_add = df_extr.merge(df_extg, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
#print(df_extr.loc[9, 'equipment_no'] == df_extg.loc[50650, 'equipment_no'])

#print(df_extr[9])
#print(df_extg[50650])

# print(len(df_add))
print()
print()
print(df_extg['cost_code_no'].dtype)
print(df_extr['cost_code_no'].dtype)
print()
print()

print(df_extg.iloc[50651] == df_extr.iloc[4])
print(df_extg.loc[50651, 'cost_code_no'])
print(df_extr.loc[4, 'cost_code_no'])
#print(df_extg.loc[9, 'date_booked'])
#print(df_extr.loc[9, 'date_booked'])




object
object


equipment_no        True
transaction_no      True
job_no              True
equipment_units     True
cost_code_no       False
date_modified       True
dtype: bool
      1001
1001.0


In [4]:
'''
2025-01-25 - Got the basic functionality down for equipment usage, and got 2022 - 2024 usage data into the DB. Note:
   - Need to move the .to_sql command down to the add
   - Need to figure out how to filter out data from INCOMING file. If we query the entire DB, all files from the DB that don't match files in the 
     incoming dataframe will get re-written. Options:
       * only keep records from INCOMING
       * find another way of comparing



'''

"\n2025-01-25 - Got the basic functionality down for equipment usage, and got 2022 - 2024 usage data into the DB. Note:\n   - Need to move the .to_sql command down to the add\n   - Need to figure out how to filter out data from INCOMING file. If we query the entire DB, all files from the DB that don't match files in the \n     incoming dataframe will get re-written. Options:\n       * only keep records from INCOMING\n       * find another way of comparing\n\n\n\n"